# Common imports & library functions

In [3]:
import doctest

# Day 1: Calorie Counting

In [21]:
def parse_calories(calories_txt):
  r"""
  >>> parse_calories("1\n2\n3\n\n4\n5")
  [[1, 2, 3], [4, 5]]
  """
  return [[int(c) for c in items_per_elf.split('\n')]
          for items_per_elf in calories_txt.split('\n\n')]

def max_calories(calories, n=1):
  """
  >>> max_calories([[1, 2, 3], [4, 5]], n=1)
  9
  >>> max_calories([[1, 2, 3], [4, 5]], n=2)
  [9, 6]
  >>> max_calories([[1000, 2000, 3000], [4000], [5000, 6000], [7000, 8000, 9000], [10000]])
  24000
  """
  sums = (sum(cs) for cs in calories)
  if n == 1:
    return max(sums)
  else:
    return sorted(sums, reverse=True)[:n]

In [22]:
doctest.run_docstring_examples(parse_calories, globs=None, verbose=True)
doctest.run_docstring_examples(max_calories, globs=None, verbose=True)

Finding tests in NoName
Trying:
    parse_calories("1\n2\n3\n\n4\n5")
Expecting:
    [[1, 2, 3], [4, 5]]
ok
Finding tests in NoName
Trying:
    max_calories([[1, 2, 3], [4, 5]], n=1)
Expecting:
    9
ok
Trying:
    max_calories([[1, 2, 3], [4, 5]], n=2)
Expecting:
    [9, 6]
ok
Trying:
    max_calories([[1000, 2000, 3000], [4000], [5000, 6000], [7000, 8000, 9000], [10000]])
Expecting:
    24000
ok


In [23]:
# Final answers
with open('day1.txt') as f:
    calories = parse_calories(f.read().strip())
    print('Part 1: ', max_calories(calories))
    print('Part 2: ', sum(max_calories(calories, 3)))

Part 1:  71506
Part 2:  209603


# Day 2: Rock Paper Scissors

In [31]:
SHAPE_SCORES = {'A': 1, 'B': 2, 'C': 3}
RESPONSES = {'X': 'A', 'Y': 'B', 'Z': 'C'}
BEATS = {('A', 'B'), ('B', 'C'), ('C', 'A')}

def bad_response(them, you):
  return RESPONSES[you]

def smart_response(them, you):
  if you == 'Y':    # tie
    return them
  elif you == 'X':  # lose
    return next(l for (l, w) in BEATS if them == w)
  elif you == 'Z':  # win
    return next(w for (l, w) in BEATS if them == l)

def shape_score(shape):
  return SHAPE_SCORES[shape]

def outcome_score(them, you):
  if them == you:  # tie
    return 3
  elif (them, you) in BEATS:  # win
    return 6
  else:  # lose
    return 0

def score_game(game, response_fn):
  them, you = game.split(' ')
  you = response_fn(them, you)
  return shape_score(you) + outcome_score(them, you)

def score_strategy(strategy, response_fn):
  r"""
  >>> score_strategy("A Y\nB X\nC Z", response_fn=bad_response)
  15
  >>> score_strategy("A Y\nB X\nC Z", response_fn=smart_response)
  12
  """
  return sum(score_game(g, response_fn) for g in strategy.split('\n'))


In [32]:
doctest.run_docstring_examples(score_strategy, globs=None, verbose=True)

Finding tests in NoName
Trying:
    score_strategy("A Y\nB X\nC Z", response_fn=bad_response)
Expecting:
    15
ok
Trying:
    score_strategy("A Y\nB X\nC Z", response_fn=smart_response)
Expecting:
    12
ok


In [33]:
# Final answers
with open('day2.txt') as f:
    strategy = f.read().strip()
    print('Part 1: ', score_strategy(strategy, bad_response))
    print('Part 2: ', score_strategy(strategy, smart_response))

Part 1:  11906
Part 2:  11186


# Day 3: Rucksack Reorganization

In [80]:
from dataclasses import dataclass
import string
from typing import List
from functools import reduce

PRIORITIES = {}

for c in string.ascii_lowercase:
  PRIORITIES[c] = ord(c) - 96
for C in string.ascii_uppercase:
    PRIORITIES[C] = ord(C) - 38

def priority(c):
  """
  >>> priority('a')
  1
  >>> priority('Z')
  52
  """
  return PRIORITIES[c]

@dataclass
class Rucksack:
  contents: str
  
  @property
  def c1(self):
    return self.contents[:len(self.contents)//2]

  @property
  def c2(self):
    return self.contents[len(self.contents)//2:]
  
  def __iter__(self):
    return iter(self.contents)

  def shared(self):
    """
    >>> Rucksack("vJrwpWtwJgWrhcsFMMfFFhFp").shared()
    'p'
    """
    return next(iter(set(self.c1) & set(self.c2)))

@dataclass
class Group:
  rucksacks: List[Rucksack]
  
  def shared(self):
    """
    >>> Group([Rucksack(r) for r in '''vJrwpWtwJgWrhcsFMMfFFhFp
    ... jqHRNqRjqzjGDLGLrsFMfFZSrLrFZsSL
    ... PmmdzqPrVvPwwTWBwg'''.splitlines()]).shared()
    'r'
    """
    common = reduce(lambda a, b: a & b, (set(r) for r in self.rucksacks))
    return next(iter(common))

  @staticmethod
  def from_rucksacks(rucksacks):
    return [Group(g) for g in zip(*([iter(rucksacks)] * 3))]

def shared_priority(objs):
  r"""
  >>> rucksacks = [Rucksack(r) for r in '''vJrwpWtwJgWrhcsFMMfFFhFp
  ... jqHRNqRjqzjGDLGLrsFMfFZSrLrFZsSL
  ... PmmdzqPrVvPwwTWBwg
  ... wMqvLMZHhHMvwLHjbvcjnnSBnvTQFn
  ... ttgJtRGJQctTZtZT
  ... CrZsJsPPZsGzwwsLwLmpwMDw'''.splitlines()]
  >>> shared_priority(rucksacks)
  157
  >>> shared_priority(Group.from_rucksacks(rucksacks))
  70
  """
  return sum(priority(o.shared()) for o in objs)
    

In [83]:
doctest.run_docstring_examples(priority, globs=None, verbose=False)
doctest.run_docstring_examples(shared_priority, globs=None, verbose=False)
doctest.run_docstring_examples(Rucksack.shared, globs=None, verbose=False)
doctest.run_docstring_examples(Group.shared, globs=None, verbose=False)

In [87]:
# Final answers
with open('day3.txt') as f:
    rucksacks = [Rucksack(cs.strip()) for cs in f]
    print('Part 1: ', shared_priority(rucksacks))
    print('Part 2: ', shared_priority(Group.from_rucksacks(rucksacks)))

Part 1:  7766
Part 2:  2415
